In [1]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from datetime import date
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MultiLabelBinarizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_array
import re

In [ ]:
df1 = pd.read_parquet("dataframes\df_final_compile.parquet")
pd.set_option('display.max_columns', None)
print(df1.shape)
df1.head(3)

In [ ]:
save1 = pd.read_parquet("dataframes\save1.parquet")
save1

In [4]:
save2 = pd.read_parquet("dataframes\save2.parquet")
save2

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
C:\Users\sulta\AppData\Local\Temp\ipykernel_24436\848422721.py:1: SyntaxWarning: invalid escape sequence '\s'
  save2 = pd.read_parquet("ignore\save2.parquet")


,id_movie,Publique cible,Epoque,Sujet principal,Type
0,73723,enfant,futuriste,animaux,dessin animé
1,315635,adulte,époque actuelle,super-héros,blockbuster
2,210577,adulte,époque actuelle,suspense,blockbuster
3,273477,adulte,futuriste,postapocalyptique,blockbuster
4,10192,enfant,époque actuelle,humour,dessin animé
...,...,...,...,...,...
2879,694234,adulte,époque actuelle,amour,film d'auteur
2880,714888,adulte,années 1970-1980,biopic,film d'auteur
2881,606954,adulte,époque actuelle,drame,film d'auteur
2882,614292,adulte,époque actuelle,drame,film d'auteur


In [5]:
save = pd.concat([save1, save2])
print(save.shape)
save

(5771, 5)


,id_movie,Publique cible,Epoque,Sujet principal,Type
0,11224,enfant,moyen-âge,amour,dessin animé
1,389,adulte,année 1950-1960,drame,film d'auteur
2,6844,adulte,antiquité,drame,blockbuster
3,665,adulte,antiquité,drame,blockbuster
4,10882,enfant,moyen-âge,amour,dessin animé
...,...,...,...,...,...
2879,694234,adulte,époque actuelle,amour,film d'auteur
2880,714888,adulte,années 1970-1980,biopic,film d'auteur
2881,606954,adulte,époque actuelle,drame,film d'auteur
2882,614292,adulte,époque actuelle,drame,film d'auteur


In [6]:
df = pd.merge(df1, save, how='left', on = ["id_movie", "id_movie"])
df

,id_movie,backdrop_path,original_language,overview,popularity_movie,poster_path,release_date,title,vote_average,vote_count,genre,gender_director,name_director,popularity_director,birthday_director,deathday_director,place_of_birth_director,movie_id,actor_name_1,actor_name_2,actor_name_3,actor_name_4,actor_name_5,actor_gender_1,actor_gender_2,actor_gender_3,actor_gender_4,actor_gender_5,actor_popularity_1,actor_popularity_2,actor_popularity_3,actor_popularity_4,actor_popularity_5,actor_birthday_1,actor_birthday_2,actor_birthday_3,actor_birthday_4,actor_birthday_5,Publique cible,Epoque,Sujet principal,Type
0,11224,/rH0DPF7pB35jxLxKb3JRUgCrrnp.jpg,en,"D’un coup de baguette magique, la fée Marraine...",79.005,/nqSJwBdvG89uHRpDDdaAy5YhogZ.jpg,1950-02-22,Cendrillon,7.041,6685,"[Family, Fantasy, Animation, Romance]",2,Hamilton Luske,2.479,1903-10-16,1968-02-19,"Chicago, Illinois, USA",11224,Ilene Woods,Eleanor Audley,Verna Felton,Claire Du Brey,Rhoda Williams,1.0,1.0,1.0,1.0,1.0,5.250,7.009,8.036,4.592,2.626,1929-05-05,1905-11-19,1890-07-20,1892-08-30,1930-07-03,enfant,moyen-âge,amour,dessin animé
1,389,/qqHQsStV6exghCM7zbObuYBiYxw.jpg,en,Un jeune homme d'origine modeste est accusé du...,58.940,/bPImGSvZtG2tvsJ9bVLrIECRDnB.jpg,1957-04-10,12 Hommes en colère,8.500,8655,[Drama],2,Sidney Lumet,4.824,1924-06-25,2011-04-09,"Philadelphia, Pennsylvania, USA",389,Martin Balsam,John Fiedler,Lee J. Cobb,E.G. Marshall,Jack Klugman,2.0,2.0,2.0,2.0,2.0,9.780,7.985,10.268,9.454,4.698,1919-11-04,1925-02-03,1911-12-08,1914-06-18,1922-04-27,adulte,année 1950-1960,drame,film d'auteur
2,6844,/rI1u4JgtnAZC99wXX4gwro4yaxe.jpg,en,Évocation de la vie de Moïse sauvé à sa naissa...,56.194,/q0KM14O75n0h4324npmThHi56FG.jpg,1956-10-05,Les Dix Commandements,7.756,1603,"[Drama, History]",2,Cecil B. DeMille,6.705,1881-08-12,1959-01-21,"Ashfield, Massachusetts, USA",6844,Charlton Heston,Yul Brynner,Anne Baxter,Edward G. Robinson,Yvonne De Carlo,2.0,2.0,1.0,2.0,1.0,22.823,8.561,9.878,9.671,15.539,1923-10-04,1920-07-11,1923-05-07,1893-12-11,1922-09-01,adulte,antiquité,drame,blockbuster
3,665,/sU3gFo7Gn67zjtKXIvzJ4omcUbH.jpg,en,"Judas Ben‐Hur, prince de Judée, retrouve son a...",56.369,/4KH7VM5YyGwxe4kLSNxix2PNTn6.jpg,1959-11-18,Ben-Hur,7.881,2747,"[History, Drama, Adventure, Action]",2,William Wyler,4.382,1902-07-01,1981-07-27,"Mülhausen, Alsace-Lorraine, German Empire [now...",665,Charlton Heston,Stephen Boyd,Hugh Griffith,Jack Hawkins,Haya Harareet,2.0,2.0,2.0,2.0,1.0,22.823,8.798,4.333,9.360,2.899,1923-10-04,1931-07-04,1912-05-30,1910-09-14,1931-09-20,adulte,antiquité,drame,blockbuster
4,10882,/1HWNGfaYB4GPwMg8piXYdQHWNuF.jpg,en,La sorcière Maléfique jette un sort à la princ...,46.057,/5GaczvmuUCxhwPNx896lo2OCIFh.jpg,1959-02-17,La Belle au bois dormant,6.900,5123,"[Fantasy, Animation, Romance, Family]",2,Clyde Geronimi,3.546,1901-06-12,1989-04-24,"Chiavenna, Lombardy, Italy",10882,Mary Costa,Bill Shirley,Eleanor Audley,Verna Felton,Barbara Luddy,1.0,2.0,1.0,1.0,1.0,4.002,2.951,7.009,8.036,3.495,1930-04-05,1921-07-06,1905-11-19,1890-07-20,1908-05-25,enfant,moyen-âge,amour,dessin animé
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5766,694234,/a8dc8eyldtHMjrUluyLRKdVIhif.jpg,it,Pendant leurs vacances d'été sur les plages de...,7.660,/waOsLWxnqA0sa00QUpZCwGL0dsS.jpg,2020-07-01,Sous le soleil de Riccione,6.200,552,"[Comedy, Romance]",2,Antonio Usbergo,0.335,None,None,None,694234,Cristiano Caccamo,Saul Nanni,Andrea Roncato,Lorenzo Zurzolo,Ludovica Martino,2.0,2.0,2.0,2.0,1.0,6.585,4.948,1.634,28.534,4.751,1989-03-21,1999-02-04,1947-03-07,2000-03-21,1997-02-26,adulte,époque actuelle,amour,film d'auteur
5767,714888,/gh1Rghpf3BIISHSAw9GsObG4TN3.jpg,es,"Argentina, 1985 est inspiré de l'histoire vrai...",10.286,/ZjOG0RZlOM6YBiVdDbINadSxS4.jpg,2022-09-29,"Argentine, 1985",7.900,617,"[Drama, History, Crime]",2,Santiago Mitre,0.674,1980-12-04,None,"Bueno

In [7]:
df_movie = df.copy()

In [8]:
date_list = ['release_date', 'birthday_director', 'deathday_director']
for date in date_list:
    try:
        df_movie[date] = pd.to_datetime(df_movie[date])
        df_movie[date] = ((df_movie[date].dt.year // 10) * 10)
    except: 
        pass

In [9]:
df_movie.head(3)

,id_movie,backdrop_path,original_language,overview,popularity_movie,poster_path,release_date,title,vote_average,vote_count,genre,gender_director,name_director,popularity_director,birthday_director,deathday_director,place_of_birth_director,movie_id,actor_name_1,actor_name_2,actor_name_3,actor_name_4,actor_name_5,actor_gender_1,actor_gender_2,actor_gender_3,actor_gender_4,actor_gender_5,actor_popularity_1,actor_popularity_2,actor_popularity_3,actor_popularity_4,actor_popularity_5,actor_birthday_1,actor_birthday_2,actor_birthday_3,actor_birthday_4,actor_birthday_5,Publique cible,Epoque,Sujet principal,Type
0,11224,/rH0DPF7pB35jxLxKb3JRUgCrrnp.jpg,en,"D’un coup de baguette magique, la fée Marraine...",79.005,/nqSJwBdvG89uHRpDDdaAy5YhogZ.jpg,1950,Cendrillon,7.041,6685,"[Family, Fantasy, Animation, Romance]",2,Hamilton Luske,2.479,1900.0,1960.0,"Chicago, Illinois, USA",11224,Ilene Woods,Eleanor Audley,Verna Felton,Claire Du Brey,Rhoda Williams,1.0,1.0,1.0,1.0,1.0,5.250,7.009,8.036,4.592,2.626,1929-05-05,1905-11-19,1890-07-20,1892-08-30,1930-07-03,enfant,moyen-âge,amour,dessin animé
1,389,/qqHQsStV6exghCM7zbObuYBiYxw.jpg,en,Un jeune homme d'origine modeste est accusé du...,58.940,/bPImGSvZtG2tvsJ9bVLrIECRDnB.jpg,1950,12 Hommes en colère,8.500,8655,[Drama],2,Sidney Lumet,4.824,1920.0,2010.0,"Philadelphia, Pennsylvania, USA",389,Martin Balsam,John Fiedler,Lee J. Cobb,E.G. Marshall,Jack Klugman,2.0,2.0,2.0,2.0,2.0,9.780,7.985,10.268,9.454,4.698,1919-11-04,1925-02-03,1911-12-08,1914-06-18,1922-04-27,adulte,année 1950-1960,drame,film d'auteur
2,6844,/rI1u4JgtnAZC99wXX4gwro4yaxe.jpg,en,Évocation de la vie de Moïse sauvé à sa naissa...,56.194,/q0KM14O75n0h4324npmThHi56FG.jpg,1950,Les Dix Commandements,7.756,1603,"[Drama, History]",2,Cecil B. DeMille,6.705,1880.0,1950.0,"Ashfield, Massachusetts, USA",6844,Charlton Heston,Yul Brynner,Anne Baxter,Edward G. Robinson,Yvonne De Carlo,2.0,2.0,1.0,2.0,1.0,22.823,8.561,9.878,9.671,15.539,1923-10-04,1920-07-11,1923-05-07,1893-12-11,1922-09-01,adulte,antiquité,drame,blockbuster


In [10]:
df_movie[['release_date', 'birthday_director', 'deathday_director']] = df_movie[['release_date', 
                                                                                 'birthday_director', 'deathday_director']].fillna(0)

In [11]:
df_movie['birthday_director'] = df_movie['birthday_director'].astype(int)
df_movie['deathday_director'] = df_movie['deathday_director'].astype(int)
df_movie.head(3)

,id_movie,backdrop_path,original_language,overview,popularity_movie,poster_path,release_date,title,vote_average,vote_count,genre,gender_director,name_director,popularity_director,birthday_director,deathday_director,place_of_birth_director,movie_id,actor_name_1,actor_name_2,actor_name_3,actor_name_4,actor_name_5,actor_gender_1,actor_gender_2,actor_gender_3,actor_gender_4,actor_gender_5,actor_popularity_1,actor_popularity_2,actor_popularity_3,actor_popularity_4,actor_popularity_5,actor_birthday_1,actor_birthday_2,actor_birthday_3,actor_birthday_4,actor_birthday_5,Publique cible,Epoque,Sujet principal,Type
0,11224,/rH0DPF7pB35jxLxKb3JRUgCrrnp.jpg,en,"D’un coup de baguette magique, la fée Marraine...",79.005,/nqSJwBdvG89uHRpDDdaAy5YhogZ.jpg,1950,Cendrillon,7.041,6685,"[Family, Fantasy, Animation, Romance]",2,Hamilton Luske,2.479,1900,1960,"Chicago, Illinois, USA",11224,Ilene Woods,Eleanor Audley,Verna Felton,Claire Du Brey,Rhoda Williams,1.0,1.0,1.0,1.0,1.0,5.250,7.009,8.036,4.592,2.626,1929-05-05,1905-11-19,1890-07-20,1892-08-30,1930-07-03,enfant,moyen-âge,amour,dessin animé
1,389,/qqHQsStV6exghCM7zbObuYBiYxw.jpg,en,Un jeune homme d'origine modeste est accusé du...,58.940,/bPImGSvZtG2tvsJ9bVLrIECRDnB.jpg,1950,12 Hommes en colère,8.500,8655,[Drama],2,Sidney Lumet,4.824,1920,2010,"Philadelphia, Pennsylvania, USA",389,Martin Balsam,John Fiedler,Lee J. Cobb,E.G. Marshall,Jack Klugman,2.0,2.0,2.0,2.0,2.0,9.780,7.985,10.268,9.454,4.698,1919-11-04,1925-02-03,1911-12-08,1914-06-18,1922-04-27,adulte,année 1950-1960,drame,film d'auteur
2,6844,/rI1u4JgtnAZC99wXX4gwro4yaxe.jpg,en,Évocation de la vie de Moïse sauvé à sa naissa...,56.194,/q0KM14O75n0h4324npmThHi56FG.jpg,1950,Les Dix Commandements,7.756,1603,"[Drama, History]",2,Cecil B. DeMille,6.705,1880,1950,"Ashfield, Massachusetts, USA",6844,Charlton Heston,Yul Brynner,Anne Baxter,Edward G. Robinson,Yvonne De Carlo,2.0,2.0,1.0,2.0,1.0,22.823,8.561,9.878,9.671,15.539,1923-10-04,1920-07-11,1923-05-07,1893-12-11,1922-09-01,adulte,antiquité,drame,blockbuster


In [12]:
# Pour récupérer le noms des acteurs dans une liste
# actor_list = []
# for col in range(1, 6): 
#     acteur_name_col = f"actor_name_{col}"
#     for i in range(0, len(df_movie)): 
#         acteur_name = df_movie.loc[i, acteur_name_col]
#         if acteur_name not in actor_list: 
#             actor_list.append(acteur_name)

In [13]:
#Pour récupérer les acteurs et la note moyenne nes films dans lequels ils ont joué

# resultat = {}

# for acteur in actor_list: 
#     index_list = []
#     popularity_list = []
#     for col in range(1, 6): 
#         acteur_name_col = f"actor_name_{col}"
        
#         index = df_movie[df_movie[acteur_name_col] == acteur].index
#         for i in index : 
#             index_list.append(i)

#     for index in index_list:
#         popularity = df_movie.loc[index, 'popularity_movie']
#         popularity_list.append(popularity)
    
#     try: 
#         resultat[acteur] = float(round(sum(popularity_list) / len(popularity_list),2))
#     except: 
#         pass


In [14]:
# Pour rajouter les colonnes avec la note moyenne des films
#     acteur_name_col = f"actor_name_{col}"
#     acteur_score_col = f"actor_score_{col}"
#     for i in range(0, len(df_movie)):
#         df_movie.loc[i, acteur_score_col] = resultat.get(df_movie.loc[i, acteur_name_col])

In [15]:
# Pour rempaler les noms des acteurs par score moyen : 
#for col in range(1, 6): 
#    acteur_name_col = f"actor_name_{col}"
#    for i in range(0, len(df_movie)): 
#        df_movie.loc[i, acteur_name_col] = resultat.get(df_movie.loc[i, acteur_name_col])



In [16]:
#df_movie = df_movie.rename(columns={'actor_name_1': 'actor_1_mean_movie', 
#                         'actor_name_2': 'actor_2_mean_movie', 
#                         'actor_name_3': 'actor_3_mean_movie',
#                         'actor_name_4': 'actor_4_mean_movie', 
#                         'actor_name_5': 'actor_5_mean_movie'
#})

In [17]:
df_movie.isnull().sum()

id_movie                     0
backdrop_path                0
original_language            0
overview                     0
popularity_movie             0
poster_path                  0
release_date                 0
title                        0
vote_average                 0
vote_count                   0
genre                        0
gender_director              0
name_director                0
popularity_director          0
birthday_director            0
deathday_director            0
place_of_birth_director    435
movie_id                     0
actor_name_1                 0
actor_name_2                 3
actor_name_3                10
actor_name_4                33
actor_name_5               242
actor_gender_1               0
actor_gender_2               3
actor_gender_3              10
actor_gender_4              33
actor_gender_5             242
actor_popularity_1           0
actor_popularity_2           3
actor_popularity_3          10
actor_popularity_4          33
actor_po

In [18]:
# Remplir les valeurs nulles dans d'autres colonnes numériques
# df_movie[['actor_popularity_2', 'actor_popularity_3', 'actor_popularity_4', 
#           'actor_popularity_5', 'actor_score_2', 'actor_score_3', 
#           'actor_score_4', 'actor_score_5']] = df_movie[['actor_popularity_2', 'actor_popularity_3', 
#                                                          'actor_popularity_4', 'actor_popularity_5', 
#                                                          'actor_score_2', 'actor_score_3', 'actor_score_4', 'actor_score_5']].fillna(0)



# Remplir les valeurs nulls dans d'autres colonnes non-numériques : 
df_movie[['actor_name_2', 
          'actor_name_3', 'actor_name_4', 'actor_name_5']] = df_movie[['actor_name_2', 
                                                                       'actor_name_3', 'actor_name_4', 'actor_name_5']].fillna('aucun')



In [19]:
df_movie.shape

(5771, 42)

In [20]:
df_movie.columns

Index(['id_movie', 'backdrop_path', 'original_language', 'overview',
       'popularity_movie', 'poster_path', 'release_date', 'title',
       'vote_average', 'vote_count', 'genre', 'gender_director',
       'name_director', 'popularity_director', 'birthday_director',
       'deathday_director', 'place_of_birth_director', 'movie_id',
       'actor_name_1', 'actor_name_2', 'actor_name_3', 'actor_name_4',
       'actor_name_5', 'actor_gender_1', 'actor_gender_2', 'actor_gender_3',
       'actor_gender_4', 'actor_gender_5', 'actor_popularity_1',
       'actor_popularity_2', 'actor_popularity_3', 'actor_popularity_4',
       'actor_popularity_5', 'actor_birthday_1', 'actor_birthday_2',
       'actor_birthday_3', 'actor_birthday_4', 'actor_birthday_5',
       'Publique cible', 'Epoque', 'Sujet principal', 'Type'],
      dtype='object')

In [21]:
# Créer un fichier parquet avec les colonnes nécéssaires pour la ML
df_movie = df_movie[['id_movie','original_language', 'popularity_movie', 'release_date', 'title',
       'vote_average', 'vote_count', 'genre', 'name_director', 'popularity_director',
       'actor_name_1', 'actor_name_2', 'actor_name_3', 'actor_name_4',
       'actor_name_5', 'Publique cible', 'Epoque', 'Sujet principal', 'Type']]

In [22]:
df_movie

,id_movie,original_language,popularity_movie,release_date,title,vote_average,vote_count,genre,name_director,popularity_director,actor_name_1,actor_name_2,actor_name_3,actor_name_4,actor_name_5,Publique cible,Epoque,Sujet principal,Type
0,11224,en,79.005,1950,Cendrillon,7.041,6685,"[Family, Fantasy, Animation, Romance]",Hamilton Luske,2.479,Ilene Woods,Eleanor Audley,Verna Felton,Claire Du Brey,Rhoda Williams,enfant,moyen-âge,amour,dessin animé
1,389,en,58.940,1950,12 Hommes en colère,8.500,8655,[Drama],Sidney Lumet,4.824,Martin Balsam,John Fiedler,Lee J. Cobb,E.G. Marshall,Jack Klugman,adulte,année 1950-1960,drame,film d'auteur
2,6844,en,56.194,1950,Les Dix Commandements,7.756,1603,"[Drama, History]",Cecil B. DeMille,6.705,Charlton Heston,Yul Brynner,Anne Baxter,Edward G. Robinson,Yvonne De Carlo,adulte,antiquité,drame,blockbuster
3,665,en,56.369,1950,Ben-Hur,7.881,2747,"[History, Drama, Adventure, Action]",William Wyler,4.382,Charlton Heston,Stephen Boyd,Hugh Griffith,Jack Hawkins,Haya Harareet,adulte,antiquité,drame,blockbuster
4,10882,en,46.057,1950,La Belle au bois dormant,6.900,5123,"[Fantasy, Animation, Romance, Family]",Clyde Geronimi,3.546,Mary Costa,Bill Shirley,Eleanor Audley,Verna Felton,Barbara Luddy,enfant,moyen-âge,amour,dessin animé
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5766,694234,it,7.660,2020,Sous le soleil de Riccione,6.200,552,"[Comedy, Romance]",Antonio Usbergo,0.335,Cristiano Caccamo,Saul Nanni,Andrea Roncato,Lorenzo Zurzolo,Ludovica Martino,adulte,époque actuelle,amour,film d'auteur
5767,714888,es,10.286,2020,"Argentine, 1985",7.900,617,"[Drama, History, Crime]",Santiago Mitre,0.674,Ricardo Darín,Peter Lanzani,Alejandra Flechner,Paula Ransenberg,Carlos Portaluppi,adulte,années 1970-1980,biopic,film d'auteur
5768,606954,it,6.531,2020,Odio l'estate,7.100,921,[Comedy],Massimo Venier,3.630,Aldo Baglio,Giovanni Storti,Giacomo Poretti,Lucia Mascino,Carlotta Natoli,adulte,époque actuelle,drame,film d'auteur
5769,614292,it,6.972,2020,Storia di vacanze,6.742,567,[Drama],Damiano D'Innocenzo,1.030,Elio Germano,Tommaso Di Cola,Giulietta Rebeggiani,Gabriel Montesi,Justin Alexander Korovkin,adulte,époque actuelle,drame,film d'auteur


ne pas utiliser pour la ML : 

'backdrop_path', 'overview', 'poster_path', 'movie_id', 'actor_birthday_1', 'actor_birthday_2',
       'actor_birthday_3', 'actor_birthday_4', 'actor_birthday_5'

id_movie, 'gender_director', 'birthday_director','deathday_director', title

'actor_gender_1', 'actor_gender_2', 'actor_gender_3', 'actor_gender_4', 'actor_gender_5'

'actor_popularity_1', 'actor_popularity_2', 'actor_popularity_3', 'actor_popularity_4', 'actor_popularity_5'

'place_of_birth_director'

In [23]:
#df_movie.to_parquet("df_movie.parquet")

In [24]:
numeric_cols = ['popularity_movie', 'vote_average', 'vote_count','popularity_director']


categorical_cols = ['original_language', 'name_director', 'Publique cible', 'Epoque',
                    'Sujet principal', 'Type', 'actor_name_1', 'actor_name_2', 'actor_name_3', 'actor_name_4', 'actor_name_5']


binarizer_cols = ['genre']

date_cols = ['release_date']

In [25]:
# Custom transformer for MultiLabelBinarizer
class MultiLabelBinarizerPipelineFriendly(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.mlb = MultiLabelBinarizer()

    def fit(self, X, y=None):
        self.mlb.fit(X)
        return self

    def transform(self, X):
        return self.mlb.transform(X)

    def get_feature_names_out(self, input_features=None):
        return self.mlb.classes_

In [26]:
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

date_transformer = Pipeline(steps=[
    ('date', OrdinalEncoder())
])

# On combine tout dans un ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols),
        ('genres', MultiLabelBinarizerPipelineFriendly(), 'genre'),
        ('date', date_transformer, date_cols)
    ]
)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)
])

In [27]:
pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['popularity_movie',
                                                   'vote_average', 'vote_count',
                                                   'popularity_director']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['original_language',
                                                   'name_director',
                                                   'Publique cible', 'Epoque',
                                                   'Sujet principal', 'Type',
                                                   'actor_name_1',
                                                   'actor_name_2',
                                                   'actor_name_3',
                                                   'actor_name_4',
                                                   'actor_name_5']),
                                                 ('genres',
                                                  MultiLabelBinarizerPipelineFriendly(),
                                                  'genre'),
                                                 ('date',
                                                  Pipeline(steps=[('date',
                                                                   OrdinalEncoder())]),
                                                  ['release_date'])]))])

In [28]:
# Transformation des données
processed_data = pipeline.fit_transform(df_movie)

# Affichage des données transformées
transformed_df = pd.DataFrame(processed_data.todense(), columns=pipeline["preprocessor"].get_feature_names_out())

In [29]:
processed_data.todense().shape

(5771, 20573)

In [30]:
processed_data.todense()

matrix([[ 6.87818062e-01,  4.20048356e-01,  1.08903903e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 4.02928019e-01,  2.46026388e+00,  1.62759563e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 3.63939329e-01,  1.41987981e+00, -3.00272967e-01, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        ...,
        [-3.41193701e-01,  5.02551931e-01, -4.86717434e-01, ...,
          0.00000000e+00,  0.00000000e+00,  7.00000000e+00],
        [-3.34932225e-01,  1.93701875e-03, -5.83493594e-01, ...,
          0.00000000e+00,  0.00000000e+00,  7.00000000e+00],
        [-3.50408435e-01, -8.80431723e-01, -4.01423191e-01, ...,
          0.00000000e+00,  0.00000000e+00,  7.00000000e+00]])

In [31]:
# Initialisation du modèle KNN
knn_model = NearestNeighbors(n_neighbors=6, metric='manhattan')  # 5 voisins les plus proches hormis le point de référence lui meme
knn_model.fit(processed_data)

NearestNeighbors(metric='manhattan', n_neighbors=6)

In [32]:
# Exemple de recommandation pour un item
def recommend_similar_films(title, 
                              data: pd.DataFrame=processed_data, 
                              model: NearestNeighbors=knn_model, 
                              original_data: pd.DataFrame=df_movie, 
                              n_neighbors: int=6) -> tuple[list[float], list[float]]:
    """
    Trouve les n éléments les plus proches pour un élément donné.

    Args:
        title: le titre d'un film
        data: Données transformées utilisées pour KNN = df processed data
        model: Modèle KNN pré-entraîné.
        original_data: Données originales (pour affichage) = df movies
        n_neighbors: Nombre de voisins à recommander = 6 pour avoir 5 recommandations

    Returns:
        films similaires: Indices et distances des filmes similaires.
    """
    index = original_data[original_data['title'] == title].index[0] # l'indice du filme recherché 
    distances, indices = model.kneighbors(data[index], n_neighbors=n_neighbors) 
    print(f"Le filme recherché : {title}:\n")
    display(pd.DataFrame(original_data.iloc[index]).T)
    display(distances[0])
    print(f"Recommandations pour le film : {title} :\n")
    display(original_data.iloc[indices[0][1:]])


In [51]:
recommend_similar_films("Jeux d'enfants")

Le filme recherché : Jeux d'enfants:



,id_movie,original_language,popularity_movie,release_date,title,vote_average,vote_count,genre,name_director,popularity_director,actor_name_1,actor_name_2,actor_name_3,actor_name_4,actor_name_5,Publique cible,Epoque,Sujet principal,Type
2778,8424,fr,10.766,2000,Jeux d'enfants,7.236,1781,"[Drama, Comedy, Romance]",Yann Samuell,1.434,Guillaume Canet,Marion Cotillard,Thibault Verhaeghe,Joséphine Lebas-Joly,Emmanuelle Grönvold,adulte,époque actuelle,amour,film français


array([ 0.        , 13.57113452, 14.01954273, 14.30619848, 14.35356091,
       14.68118888])

Recommandations pour le film : Jeux d'enfants :



,id_movie,original_language,popularity_movie,release_date,title,vote_average,vote_count,genre,name_director,popularity_director,actor_name_1,actor_name_2,actor_name_3,actor_name_4,actor_name_5,Publique cible,Epoque,Sujet principal,Type
2660,6691,fr,7.575,2000,Hors de prix,6.400,726,"[Drama, Comedy, Romance]",Pierre Salvadori,0.767,Gad Elmaleh,Audrey Tautou,Marie-Christine Adam,Vernon Dobtcheff,Jacques Spiesser,adulte,époque actuelle,amour,film français
2651,2266,fr,11.585,2000,"Paris, je t'aime",6.768,749,"[Drama, Romance]",Bruno Podalydès,2.130,Steve Buscemi,Natalie Portman,Willem Dafoe,Maggie Gyllenhaal,Axel Kiener,adulte,époque actuelle,amour,film français
2237,6615,en,19.831,2000,Une fiancée pas comme les autres,7.185,1803,"[Comedy, Drama, Romance]",Craig Gillespie,2.411,Ryan Gosling,Emily Mortimer,Paul Schneider,R.D. Reid,Kelli Garner,adulte,époque actuelle,amour,film français
4881,206408,fr,9.734,2010,Casse-tête chinois,6.591,678,"[Comedy, Drama, Romance]",Cédric Klapisch,2.755,Romain Duris,Audrey Tautou,Cécile de France,Kelly Reilly,Sandrine Holt,adulte,époque actuelle,amour,film français
4841,583268,fr,6.067,2010,Deux moi,6.400,647,"[Drama, Romance, Comedy]",Cédric Klapisch,2.755,François Civil,Ana Girardot,Camille Cottin,François Berléand,Simon Abkarian,adulte,époque actuelle,amour,film français


In [34]:
df1['title'].sample(10)

2956          Les Cinq Légendes
2850               Interstellar
2419                   D.E.B.S.
342           La Malédiction II
4426    Return to Jurassic Park
20         La Guerre des mondes
2199                     Slevin
187        Le Gendarme se marie
952          Les 101 Dalmatiens
1620              Les Infiltrés
Name: title, dtype: object

In [35]:
df1.loc[1528, 'title']

'Pirates des Caraïbes\xa0: Le Secret du coffre maudit'

Fast and Furious
Avatar
Forrest Gump
E.T. l'extra-terrestre
Terminator